# Importing the required python libraries

In [1]:
# import numpy package for arrays and stuff 
import numpy as np  
import csv
# import matplotlib.pyplot for plotting our result 
import matplotlib.pyplot as plt 
import seaborn as sb # used to plot the heatmap
# import pandas for importing csv files and data fromatting
import pandas as pd  
import pickle
# import the regressor 
from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split # used to split the data into training set (70%) and testing set (30%)
from sklearn.preprocessing import OneHotEncoder # Used to label encoding and one-hot encoding
from sklearn.model_selection import KFold

# Data loading and preprocessing

 


In [2]:
# The data is read as a pandas data frame
data_consolidated = pd.read_excel("newdatafile.xlsx", sheet_name=None)


In [3]:
frames =[keys for keys in data_consolidated]
data = pd.concat([data_consolidated['0'], data_consolidated['1']])

In [4]:
data = data_consolidated['0']
for i in range(1,len(data_consolidated)):
    data = pd.concat([data, data_consolidated[str(i)]])

In [5]:
data_org=data

In [6]:
data_4 = np.zeros((0,25))
for i in range(1295):
    tmp1 = data.iloc[((i*8760)+1416):((i*8760)+2160),:]
    tmp2 = data.iloc[((i*8760)+3624):((i*8760)+4344),:]
    tmp3 = data.iloc[((i*8760)+5832):((i*8760)+6552),:]
    tmp4 = data.iloc[((i*8760)+8016):((i*8760)+8760),:]
    
    data_4 = np.concatenate((data_4,tmp1,tmp2,tmp3,tmp4), axis=0)

In [7]:
data = data_4

In [8]:
data = pd.DataFrame(data, columns = data_org.columns)

In [9]:
data=data.drop(['Unnamed: 0', 'in:Date/Time', 'in:Hour', 'in:Building 02', 'in:Building 03','in:AF Shading State', 
                'out:Heating Loads - KW/m2', 'out:Lighting Loads - KW/m2', 'out:Equipments Loads - KW/m2 ',
                'out:Total Cooling Loads - KW/m2', 'out:Total Heating Loads - KW/m2',
                'out:Total Lighting Loads - KW/m2',
                'out:Total Equipments Loads - KW/m2', 'out:Total EUI - KW/m2'
               ], axis=1)

In [10]:
hours=np.array([range(1,25)])

hours=hours.T
hours.shape
H=np.tile(hours,[157990,1])
data['in:Hour'] = H
print(data.columns)

Index(['in:Orientation', 'in:Building 00', 'in:Building 01',
       'in:Facade level Height', 'in:Glazing Type - U-value W/m2K',
       'in:Exterior Wall - U-value W/m2K', 'in:AF-(Opening Percentage)',
       'in:AF-Shade Factor', 'in:Solar Radiation W/m2',
       'in:Operative Temperature', 'out:Cooling Loads - KW/m2', 'in:Hour'],
      dtype='object')


In [11]:
months1 = np.array([3])
months1 = np.tile(months1, [744,1])
months2 = np.array([6])
months2 = np.tile(months2, [720,1])
months3 = np.array([9])
months3 = np.tile(months3, [720,1])
months4 = np.array([12])
months4 = np.tile(months4, [744,1])
months = np.concatenate((months1, months2,months3, months4), axis=0)
months = np.tile(months, [1295,1])
months.shape
data['in:Month'] = months

In [12]:
daytype1 = np.zeros((1,1))
for i in range(30):
    t = np.repeat(i+1,24)
    t = np.reshape(t,(t.shape[0],1))
#     print(t)
#     print(t.shape)
    daytype1 = np.concatenate((daytype1, t  ), axis=0)

daytype1 = daytype1[1:,:]
t = np.repeat(31,24)
t = np.reshape(t,(t.shape[0],1))
daytype2 = np.concatenate((daytype1, t), axis=0) 

day = np.concatenate((daytype1, daytype2, daytype2, daytype1), axis=0)
day = np.tile(day, [1295,1])
data['in:day'] = day
data

,in:Orientation,in:Building 00,in:Building 01,in:Facade level Height,in:Glazing Type - U-value W/m2K,in:Exterior Wall - U-value W/m2K,in:AF-(Opening Percentage),in:AF-Shade Factor,in:Solar Radiation W/m2,in:Operative Temperature,out:Cooling Loads - KW/m2,in:Hour,in:Month,in:day
0,0,Low,Low,12,SingleG0,0,0.0,1.0,0.0,17.428003,0.0,1,3,1.0
1,0,Low,Low,12,SingleG0,0,0.0,1.0,0.0,17.150332,0.0,2,3,1.0
2,0,Low,Low,12,SingleG0,0,0.0,1.0,0.0,16.914902,0.0,3,3,1.0
3,0,Low,Low,12,SingleG0,0,0.0,1.0,0.0,16.69424,0.0,4,3,1.0
4,0,Low,Low,12,SingleG0,0,0.0,1.0,0.0,16.474319,0.0,5,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791755,3,High,High,60,TripleGlz-Krypton Filled,2,0.0,1.0,0.0,19.278899,0.0,20,12,30.0
3791756,3,High,High,60,TripleGlz-Krypton Filled,2,0.0,1.0,0.0,19.17135,0.0,21,12,30.0
3791757,3,High,High,60,TripleGlz-Krypton Filled,2,0.0,1.0,0.0,19.070901,0.0,22,12,30.0
3791758,3,High,High,60,TripleGlz-Krypton Filled,2,0.0,1.0,0.0,18.967557,0.0,23,12,30.0


### One-hot encoding of categorical features

In [13]:
# Array that specifies which columns are categorical inputs
categorical_features = [ 'in:Orientation', 'in:Building 00',
       'in:Building 01', 'in:Glazing Type - U-value W/m2K',
       'in:Exterior Wall - U-value W/m2K', 'in:Hour', 'in:Month', 'in:day',] # Except column "7", all are label encoded and "7" is 
                                                               # one-hot encoded. 
# Array that specifies which columns are continuous inputs
contnuous_features = ['in:Facade level Height','in:AF-(Opening Percentage)', 'in:AF-Shade Factor',
       'in:Solar Radiation W/m2', 'in:Operative Temperature']
output_feature = 'out:Cooling Loads - KW/m2'  # specifies which column is output


temp={}
label_encoders = {} 
# for loop starts for label encoding
for i in range(0,len(categorical_features)):
    #codes for one-hot encoding
    label_encoders[categorical_features[i]] = OneHotEncoder(sparse = 'True',handle_unknown='ignore')
    label_encoders[categorical_features[i]].fit(data[categorical_features[i]].values.reshape(-1,1))
    temp[categorical_features[i]]=label_encoders[categorical_features[i]].transform(data[categorical_features[i]].values.reshape(-1,1)).toarray() 

In [14]:
#printing the dimensions of each categorical variables after one-hot encoding
for i in range(0,len(temp)):
    print(temp[categorical_features[i]].shape)

(3791760, 4)
(3791760, 3)
(3791760, 3)
(3791760, 4)
(3791760, 3)
(3791760, 24)
(3791760, 4)
(3791760, 31)


In [15]:
# Adding the one hot encoded features into the data
# The new columns will be named from 13 onwards.
ind=data.shape[1]+1
for i in range(0,len(categorical_features)):
    for j in range(0,temp[categorical_features[i]].shape[1]):
        data[str(ind)] = temp[categorical_features[i]][:,j]
        ind=ind+1
    data=data.drop([categorical_features[i]], axis=1) # The old column is deleted from the data
      
    

In [16]:
print(data) # Data after one-hot encoding.
print(data.shape)     # New dimension of the data. 

        in:Facade level Height in:AF-(Opening Percentage) in:AF-Shade Factor  \
0                           12                        0.0                1.0   
1                           12                        0.0                1.0   
2                           12                        0.0                1.0   
3                           12                        0.0                1.0   
4                           12                        0.0                1.0   
...                        ...                        ...                ...   
3791755                     60                        0.0                1.0   
3791756                     60                        0.0                1.0   
3791757                     60                        0.0                1.0   
3791758                     60                        0.0                1.0   
3791759                     60                        0.0                1.0   

        in:Solar Radiation W/m2 in:Oper

# Modeling with Decision Tree for Regression

In [17]:
# Variable X will seperate the input variables from data
X = data.values # First 11 columns are taken 
# X=np.concatenate((X,data.iloc[:, 4:].values),axis=1) # "or1","or2,"or3" and "or4" are appended to first 11 columns
# y = data.iloc[:, 3].values # output column is stored into variable y
print("The input data....")
# print(X[:,0:5])
print(X.shape)
# print("The output data")
# print(y)
# print(y.shape)

The input data....
(3791760, 82)


In [18]:
index = np.random.permutation(data.shape[0])
np.savetxt("indexRF.csv", index)

In [19]:
filename = 'indexRF.csv'
raw_data = open(filename, 'rt')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
x = list(reader)
index = np.array(x).astype('float')

In [20]:
X=data.values
instances=X.shape[0]
# index = np.random.permutation(int(instances))
train_data=np.zeros((int(np.floor(0.8*instances)), X.shape[1]))
# print(train.shape)
 

for i in range(int(np.floor(0.8*instances))):
    train_data[i,:] = X[int(index[i]),:]
 

print(train_data.shape)
t4=int(instances)-int(np.floor(0.8*instances))
test_new=np.zeros((t4, data.shape[1]))
 
t3=int(np.floor(0.8*instances))

for i in range(t4):
    test_new[i,:] = X[int(index[i+t3]),:]
 
print(test_new.shape)    

(3033408, 82)
(758352, 82)


In [21]:
n_splits = 5
kf = KFold(n_splits)
kf.get_n_splits(train_data) 
KFold(n_splits=5, random_state=None, shuffle=False)
trees=[20, 40, 60, 80, 100, 10, 30, 50, 70, 90]
# trees= [10, 20]
ccp_alphas = [0, 0.010, 0.020, 0.030]
# ccp_alphas =[0, 0.010]
bootstrap_opt = [True, False]
Results = np.zeros((len(trees)*len(bootstrap_opt), n_splits,3))
p=0
for tree in trees: 
    for bs in bootstrap_opt:
        #for alpha in ccp_alphas:
            q=0
            for train_index, test_index in kf.split(train_data):
                print("TRAIN:", train_index, "TEST:", test_index)
                X_train, X_test = train_data[train_index], train_data[test_index]
         
                train = X_train[:,0:len(contnuous_features)] # First 11 columns are taken
                train_y = X_train[:,len(contnuous_features)]
                train = np.concatenate((train,X_train[:, (len(contnuous_features)+1):]),axis=1) 
        
                test = X_test[:,0:len(contnuous_features)] # First 11 columns are taken
                test_y = X_test[:,len(contnuous_features)]
                test = np.concatenate((test,X_test[:, (len(contnuous_features)+1):]),axis=1)  
        
                regr = RandomForestRegressor(n_estimators=tree, bootstrap = bs, ccp_alpha = 0, n_jobs=-1)
                regr.fit(train, train_y)
                y_pred = regr.predict(test) # Prediction corresponding to the test set
                print(p,q)
                Results[p,q,0] = mean_squared_error(test_y, y_pred)
                Results[p,q,1] = mean_absolute_error(test_y, y_pred)
                Results[p,q,2] = r2_score(test_y, y_pred)
            
                q+=1
            p+=1


TRAIN: [ 606682  606683  606684 ... 3033405 3033406 3033407] TEST: [     0      1      2 ... 606679 606680 606681]
0 0
TRAIN: [      0       1       2 ... 3033405 3033406 3033407] TEST: [ 606682  606683  606684 ... 1213361 1213362 1213363]
0 1
TRAIN: [      0       1       2 ... 3033405 3033406 3033407] TEST: [1213364 1213365 1213366 ... 1820043 1820044 1820045]
0 2
TRAIN: [      0       1       2 ... 3033405 3033406 3033407] TEST: [1820046 1820047 1820048 ... 2426724 2426725 2426726]
0 3
TRAIN: [      0       1       2 ... 2426724 2426725 2426726] TEST: [2426727 2426728 2426729 ... 3033405 3033406 3033407]
0 4
TRAIN: [ 606682  606683  606684 ... 3033405 3033406 3033407] TEST: [     0      1      2 ... 606679 606680 606681]
1 0
TRAIN: [      0       1       2 ... 3033405 3033406 3033407] TEST: [ 606682  606683  606684 ... 1213361 1213362 1213363]
1 1
TRAIN: [      0       1       2 ... 3033405 3033406 3033407] TEST: [1213364 1213365 1213366 ... 1820043 1820044 1820045]
1 2
TRAIN: [    

13 1
TRAIN: [      0       1       2 ... 3033405 3033406 3033407] TEST: [1213364 1213365 1213366 ... 1820043 1820044 1820045]
13 2
TRAIN: [      0       1       2 ... 3033405 3033406 3033407] TEST: [1820046 1820047 1820048 ... 2426724 2426725 2426726]
13 3
TRAIN: [      0       1       2 ... 2426724 2426725 2426726] TEST: [2426727 2426728 2426729 ... 3033405 3033406 3033407]
13 4
TRAIN: [ 606682  606683  606684 ... 3033405 3033406 3033407] TEST: [     0      1      2 ... 606679 606680 606681]
14 0
TRAIN: [      0       1       2 ... 3033405 3033406 3033407] TEST: [ 606682  606683  606684 ... 1213361 1213362 1213363]
14 1
TRAIN: [      0       1       2 ... 3033405 3033406 3033407] TEST: [1213364 1213365 1213366 ... 1820043 1820044 1820045]
14 2
TRAIN: [      0       1       2 ... 3033405 3033406 3033407] TEST: [1820046 1820047 1820048 ... 2426724 2426725 2426726]
14 3
TRAIN: [      0       1       2 ... 2426724 2426725 2426726] TEST: [2426727 2426728 2426729 ... 3033405 3033406 3033407

In [22]:
np.savetxt('RF_Cross_validation_MSE_results1.csv', Results[:,:,0], delimiter=',')
np.savetxt('RF_Cross_validation_MAE_results1.csv', Results[:,:,1], delimiter=',')
np.savetxt('RF_Cross_validation_R2_results1.csv', Results[:,:,2], delimiter=',')

In [23]:
0%4

0

In [24]:
np.savetxt('RF_Cross_validation_MSE_results1.csv', Results[:,:,0], delimiter=',')
np.savetxt('RF_Cross_validation_MAE_results1.csv', Results[:,:,1], delimiter=',')
np.savetxt('RF_Cross_validation_R2_results1.csv', Results[:,:,2], delimiter=',')
RMSE_summary = Results[:,:,0]
RMSE_summary = np.sum(RMSE_summary, axis = 1)
print(RMSE_summary)
RMSE_summary = np.argmin(RMSE_summary)
print(RMSE_summary)

ind = RMSE_summary//10
tree = trees[ind]

if RMSE_summary%2 == 0:
    bs = True
else:
    bs = False

[1.11414836e-07 1.92391918e-07 1.06390933e-07 1.91350502e-07
 1.05549456e-07 1.91440714e-07 1.04243895e-07 1.91398991e-07
 1.03216978e-07 1.91496689e-07 1.24065120e-07 1.93085234e-07
 1.09388316e-07 1.91429393e-07 1.05578979e-07 1.91517721e-07
 1.04775117e-07 1.91478603e-07 1.04324074e-07 1.91172172e-07]
8


In [25]:
print(RMSE_summary)
print(tree,bs)

8
20 True


In [26]:
RMSE_summary

8

In [28]:
train = train_data[:,0:len(contnuous_features)] # First 11 columns are taken
train_y = train_data[:,len(contnuous_features)]
train = np.concatenate((train,train_data[:, (len(contnuous_features)+1):]),axis=1)  

test = test_new[:,0:len(contnuous_features)] # First 11 columns are taken
test_y = test_new[:,len(contnuous_features)]
test = np.concatenate((test,test_new[:, (len(contnuous_features)+1):]),axis=1)  

 
regr = RandomForestRegressor(n_estimators=tree, bootstrap = bs)
regr.fit(train, train_y)

y_pred = regr.predict(test) # Prediction corresponding to the test set
y_pred=y_pred.reshape(y_pred.shape[0],1) # Reshaping y_pred for visualization
test_y=test_y.reshape(test_y.shape[0],1) # The actual values of the test set
result=np.concatenate((test_y,y_pred),axis=1) # Actual value and predicted value is stored in "result" variable.

np.savetxt("RF_result.csv",result)
print("Final Results")
print("RMSE: ")
print( mean_squared_error(test_y, y_pred))
print("MAE: ")      
print(mean_absolute_error(test_y, y_pred))
print("R2 score: ")
print(r2_score(test_y, y_pred))

Final Results
RMSE: 
1.986078835039573e-08
MAE: 
3.1688454368227625e-05
R2 score: 
0.9998538206181501


In [29]:
np.savetxt("test_data.csv", test_new, delimiter=",")

In [30]:
#Model saving
filename = 'DT_finalized_model1.sav'
pickle.dump(regr, open(filename, 'wb'))

In [31]:
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(test)
y_pred=y_pred.reshape(y_pred.shape[0],1) # Reshaping y_pred for visualization
test_y=test_y.reshape(test_y.shape[0],1) # The actual values of the test set
result=np.concatenate((test_y,y_pred),axis=1) # Actual value and predicted value is stored in "result" variable.
print( mean_squared_error(test_y, y_pred))

1.986078835039573e-08


In [32]:
RES = np.concatenate((test_y,y_pred),axis=1)
RES.shape

(758352, 2)

In [33]:
np.savetxt("REsult1.csv", RES)